In [1]:
# import packages
import glob
from pathlib import Path
import os
import numpy as np
from datetime import datetime
from datetime import timedelta
import pandas as pd
import calendar
import geopandas as gpd
import cartopy
import matplotlib.pyplot as plt
import math
from pathos.threading import ThreadPool as Pool
from scipy.optimize import least_squares
import sklearn
from sklearn.linear_model import LinearRegression
from scipy.stats import gaussian_kde
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
import random
from scipy.optimize import minimize

In [2]:
# import python functions
from f_snow_module import *

In [3]:
work_dir=Path('/scratch/fransjevanoors/global_sr')
#work_dir=Path('/tudelft.net/staff-umbrella/LSM root zone/global_sr')
data_dir=Path(f'{work_dir}/data')
out_dir = Path(f"{work_dir}/output")

print(work_dir)
print(out_dir)
print(data_dir)


/scratch/fransjevanoors/global_sr
/scratch/fransjevanoors/global_sr/output
/scratch/fransjevanoors/global_sr/data


### 1. Select catchments with snow

In [ ]:
# select catchments with temperature <0 AND precipitation>1, because if no p there is no snow possible
catch_id_list = np.genfromtxt(f'{work_dir}/output/gsim_aus_catch_id_list_lo_sel.txt',dtype='str')[:]
snow_catch=[]
for catch_id in catch_id_list:
    a = pd.read_csv(f'{work_dir}/output/forcing_timeseries/processed/daily/{catch_id}_1981_2010.csv',index_col=0)
    t = a[(a.tas<0)&(a.p>1)]
    if (len(t))>0:
        if (len(t)>0.05*len(a)):
            snow_catch.append(catch_id)
np.savetxt(f'{work_dir}/output/snow/catch_id_list_snow_t_and_p.txt',snow_catch,fmt='%s')

In [4]:
snow_id_list = np.genfromtxt(f'{work_dir}/output/snow/catch_id_list_snow_t_and_p.txt',dtype='str')[:]
snow_id_list

array(['ar_0000023', 'ar_0000024', 'ar_0000025', ..., 'us_0008723',
       'us_0008726', 'us_0008766'], dtype='<U10')

### 2. get timeseries of liquid precipitation

In [ ]:
catch_id_list = np.genfromtxt(f'{work_dir}/output/snow/catch_id_list_snow_t_and_p.txt',dtype='str')[0:3]

# run in parallel
# make lists for parallel computation
catch_list = catch_id_list
work_dir_list = [work_dir] * len(catch_list)
run_function_parallel_snow(catch_list, work_dir_list)

In [5]:
len(snow_id_list)

2811

In [8]:
catch_id_list = np.genfromtxt(f'{work_dir}/output/snow/catch_id_list_snow_t_and_p.txt',dtype='str')


In [9]:
# check catchments that are not processed yet
catch_id_list_lo_done = []
for filepath in glob.iglob(f'{out_dir}/snow/timeseries/*'):
    f = os.path.split(filepath)[1] # remove full path
    f = f[:-4] # remove .csv extension
    catch_id_list_lo_done.append(f)

dif_list=list(set(catch_id_list)-set(catch_id_list_lo_done))
dif_list

2247


['fi_0000105',
 'ca_0005949',
 'us_0008473',
 'ca_0003447',
 'us_0000011',
 'ca_0006085',
 'sk_0000031',
 'de_0000661',
 'at_0000011',
 'ca_0000025',
 'ca_0005660',
 'ca_0004070',
 'us_0002624',
 'ca_0005463',
 'fi_0000135',
 'ca_0004993',
 'no_0000177',
 'ca_0004984',
 'ca_0004824',
 'no_0000004',
 'ca_0003176',
 'us_0004713',
 'ca_0003446',
 'ca_0003665',
 'ca_0004615',
 'ca_0004170',
 'no_0000147',
 'ca_0000748',
 'ca_0001995',
 'ca_0005534',
 'de_0000085',
 'ch_0000026',
 'ca_0006268',
 'ca_0004550',
 'ca_0001977',
 'us_0004702',
 'us_0000414',
 'ca_0000319',
 'ca_0003232',
 'ca_0005906',
 'ru_0000099',
 'ca_0002351',
 'ca_0005903',
 'ca_0001461',
 'ca_0002388',
 'no_0000016',
 'sk_0000024',
 'us_0006683',
 'us_0003645',
 'fi_0000062',
 'us_0000803',
 'ru_0000365',
 'by_0000010',
 'ch_0000063',
 'ca_0004310',
 'ca_0003337',
 'us_0003658',
 'ch_0000078',
 'ca_0004366',
 'us_0000004',
 'ru_0000414',
 'ca_0003016',
 'ch_0000095',
 'ca_0003712',
 'us_0004839',
 'ca_0003486',
 'ca_00019